### 一. 使用缓存的目的是什么？
#### 1. 高性能（缓存的数据要求一段时间内不发生变化）
 * 不使用用缓存：一个用户访user1问查询接口， 接口直接查询数据库（加入耗时800毫秒）；此后user2再次查询同一条数据， 则又会耗时800毫秒；假设数据在10分钟内没有变化，10分钟内有1000个用户查询的话， 就会耗时1000个800毫秒；
 * 使用缓存后： 第一次查询时，缓存中没有，则需要查询数据库，耗时800毫秒；但由于数据在十分钟内没有变化，因此此后的999次查询都可以从内存中直接查询， 每次耗时10ms，共耗时999个10毫秒；
 * 或者某些复杂查询可能很耗时，缓存后可以直接查出
 
#### 2. 高并发  
* 数据库如果每秒4000个请求，则数据库可能会宕机；   
 如果对每个query，先在缓存中查询，查不到再使用数据库查询；这种方法会让4000个请求分散成3000次缓存查询+1000次数据库查询
* 为什么缓存比数据库更能支持高并发？   
 因为缓存是内存查询，支持高并发查询；而数据库需要磁盘，一般建议每秒查询次数一般不超过2000次

### 三. 为什么redis这种单线程模型比多线程的memcache有更高的性能？
#### 1. redis与memcached的区别？  
   redis支持的数据类型比memcached更多，更能支持复杂应用     
   memcached没有原声的分布式集群， 需要客户端进行分片后再写；redis原生支持cluster模式 
#### 2. **redis的线程模型？**   
1. redis基于io多路复用的网络模型进行IO, 使用一个叫做"文件事件处理器"的东西处理接受连接,可读,可写,关闭事件:  
   即可理解为serversocketChannel注册了4种状态的selectionkey, 用selector轮询检查
    * key.isReadable()
    * key.isWriteable()
    * key.isAccept()
    * key.isClose()
2. 上面注册的4种状态的socket会被放入对应的四种队列中, selector选择一种状态的socket后, 文件事件处理器的4个不同部分会做出处理, 由于文件事件处理器是单线程的, 即处理完1个socket才能处理下一个socket, 所以redis是单线程的  
    
### 四. redis有哪些数据类型, 有哪些应用场景
(面向初级开发者)  
* string
* list
* set
* zset : sortedset, 相当于map结构, 但是对每个value赋予一个score权重, 同一个key, 不同的value不会被覆盖, 而是按照score排序存储

### 五. 如何实现redis的过期策略
#### 1. redis中的数据会丢失?  
  redis是一个缓存, 而不是一个数据库, 既然是缓存, 当数据超过内存限制, 自然会丢弃一些过期数据
  
#### 2. 对某个key设置过期时间后, 数据在这个时间后仍然存在?  
    1. 这涉及到redis的一个内部逻辑, 如何删除那些设置了过期时间的key?      
    采用2种删除方式, 定期删除+惰性删除  
        * 定期删除: redis每隔100ms, 从所有设置超时时间的key中抽样一部分进行检查, 删除过期的kv对.(抽样检查的目的是, redis中可能存在大量设置过期时间的key, 如果全部检查会导致cpu负载全部花在检查上了)
        * 惰性删除: 当查询1个key时, 如果redis发现设置了过期时间且已经过期, 则会删除这个key并返回空
        
    2. 如上删除策略, 如果一些key既没有被定期删除采样到, 也没有在一段时间内继续访问, 那该如何删除呢?    
    这部分数据会在内存淘汰机制中被删除, 内存淘汰机制主要有以下几种(一般都是使用allkeys-lru机制)
        * allkeys-lru : 对所有key进行LRU
        * volatile-lru : 只对设置了过期时间的key进行lru, 淘汰最近最久未被使用的
        * noevication : 内存慢了以后在插入数据直接报错
        * allkeys-rendom : 选择一个key随机删除   
    
    以上需要在 redis.conf中配置最大内存使用大小, 然后配置内存淘汰策略   
    3. 如何实现一个LRU缓存(双向链表+HashTable)

### 六. 生产环境下, redis如何实现高并发高可用机制?
#### 1. redis如何承载高并发? 实现每秒10万+的QPS (**master-slave读写分离结构**)?  
    1. 单节点的redis, qps一般在1万左右, 想承载10万qps.必须上redis的读写分离模式  
    2. qps是每秒处理的查询数量, 就表示这是一个读多写少的场景, 也正是缓存应用的场景; 如果是写多读少, 就无法使用分布式缓存了, 需要使用一些列复杂的异步消息队列实现. 
    3. redis的读写分离是这样的, 使用单个master节点负责写, master会向slave节点异步同步数据,其余slave节点负责读; 这种架构的好处在于, 对于处理读请求的slave节点可以水平扩容来满足qps; 
    <img src="img/redis1.png" width="60%">
**redis的各种架构演练**    
`redis replication` -> `redis 主从架构` -> `redis 读写分离架构` -> `水平扩容支持高并发`  

#### 2. redis replication:  
    1. redis master节点采用异步方式向slave节点复制数据; 不过从redis2.0开始, slave节点也会定期向master确认自己的数据复制量  
    2. 1个redis master节点, 可以配置多个slave节点
    3. slave节点也可以连接其它slave节点
    4. slave在复制master的数据时, 不会block自己的查询请求, 只不过此事的查询请求是在老的数据集上进行的;   
      但是slave复制数据完成之后, 需要先删除旧数据集, 加载新数据集, 此时会block查询请求  
    5. slave 节点主要用来横向扩容, 做读写分离, 提高qps
    6. master节点为什么要开启持久化, 它对安全性的意义是什么?   
      因为如果master不开启持久化, 当master宕机后, 再次重启会发现当前没有数据, 此时mster向slave同步的数据也是空的, 这会导致原本存在slave上的数据也被100%丢失

#### 3. redis数据丢失的两种情况  
    1. 异步数据复制导致数据丢失
      因为master数据时一步复制的, 如果slave复制的延迟很高, 而此时master宕机, 就会导致master的数据没来得急复制到slave而丢失  
      
    2. 脑裂, 导致多个master存在  
      脑裂是因为网络分区而产生的, 网络分区后, sentinal和slave在一个分区, master自己在一个分区, 此时sentinal会把另外一个slave提升成mstaer, 而原来还存在1个master可以负责读, 导致2个master数据不一致.
    
    3. 参数设置减轻`数据异步复制`和`脑裂`导致的数据丢失问题  
       有2个参数可供修改:     
       min-slaves-to-write : 从节点小于多少个，主节点就不写了
       min-slaves-max-lag  : 从节点大于多少秒延迟，就不写了

#### 4. (Sentinal)底层原理 ？？



#### 数据分布算法
